# -2) imports

In [27]:
from google.cloud import datastore
import pandas as pd
from vertexai.generative_models import GenerativeModel
import vertexai.preview.generative_models as generative_models

# -1) helper functions

In [160]:
def generate(input_array, max_output_tokens=8192, temperature=1, top_p=0.95):
    model = GenerativeModel("gemini-1.5-pro-preview-0409")
    params = {}
    params["safety_settings"] = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }
    params["generation_config"] = {"max_output_tokens": max_output_tokens,"temperature": temperature,"top_p": top_p}
    params["stream"] = True
    ##responses = model.generate_content(input_array,generation_config=generation_config,safety_settings=safety_settings, stream=True)
    responses = model.generate_content(input_array,**params)
    return responses

# 0) Settings

In [164]:
query = """
    SELECT 
        jsonPayload.queryresult.text,
        jsonPayload.queryresult.responsemessages[0].text.text,
        jsonPayload.queryresult.diagnosticinfo.session_id,
        timestamp
    FROM 
        `gen-ai-389408.questionnaire_ai_du_var.dialogflow_runtime_googleapis_com_requests_20240422`
    WHERE
        jsonPayload.queryresult.diagnosticinfo.session_id IS NOT NULL
    ORDER BY 
        timestamp
"""

# 1) code

## 1.1) query

In [162]:
query_job = client.query(query)

## 1.2) analyse results

In [163]:
data = []
for results in query_job:
    result = {  "question"          : results[0]    ,
                "responsemessages"  : results[1][0] ,
                "session_id"        : results[2]    ,
                "timestamp"         : results[3]    }
    data.append(result)

BadRequest: 400 Array index 0 is out of bounds (overflow)

Location: US
Job ID: 655c2ac1-24c4-4061-92ba-cd545ef2416f


In [131]:
df = pd.DataFrame.from_dict(data)

In [132]:
df

,question,responsemessages,session_id,timestamp
0,bonjour,"Bonjour, je suis là pour collecter des cas d'u...",dfMessenger-d6b48432-b962-4ada-93f2-2b0defc37430,2024-04-22 12:35:48.106673+00:00
1,bonjour,"Bonjour, je suis là pour collecter des cas d'u...",dfMessenger-ae908230-9976-4426-aab9-41f1508f41b5,2024-04-22 12:47:26.205971+00:00
2,Dans la peau de John malkovitch,"Je suis désolé, je ne comprends pas. Pourriez-...",dfMessenger-aa9957cd-c6d5-4002-b364-d1e1caad59d7,2024-04-22 12:48:04.715370+00:00
3,Bonjour,"Bonjour, je suis là pour collecter des cas d'u...",dfMessenger-3f4abed7-9a2d-4ed4-8b59-e3c678778561,2024-04-22 12:48:19.661280+00:00
4,bONJOUR,"Bonjour, je suis là pour collecter des cas d'u...",dfMessenger-0ae41a4d-6864-49f2-bf08-99d4d803df98,2024-04-22 12:48:21.351034+00:00
5,"Bonjour, posez moi une question","Bonjour, je suis là pour collecter des cas d'u...",dfMessenger-5eb176e3-eb98-4149-97da-210390464910,2024-04-22 12:48:22.556217+00:00
6,Peus-tu m'aider à préparer un démonstration de...,"Bien sûr, je peux vous aider à préparer une dé...",dfMessenger-aa9957cd-c6d5-4002-b364-d1e1caad59d7,2024-04-22 12:48:32.305667+00:00
7,directeur adjoint,Très bien. Pourriez-vous me dire en quoi consi...,dfMessenger-3f4abed7-9a2d-4ed4-8b59-e3c678778561,2024-04-22 12:48:35.604114+00:00
8,Je suis chef de cuisine dans un collège,D'accord. Pensez-vous que l'IA pourrait être u...,dfMessenger-5eb176e3-eb98-4149-97da-210390464910,2024-04-22 12:48:39.699706+00:00
9,Oui,Pourriez-vous me donner un exemple de cas d'us...,dfMessenger-5eb176e3-eb98-4149-97da-210390464910,2024-04-22 12:48:47.534093+00:00


## 1.3) reconstruct the conversation

In [133]:
prompt = """
Voici une conversation entre un chatbot qui doit trouver des cas d'usages pour l'IA générationnelle
en discutant avec des utilisateurs. 
Regarde la conversation ci jointe, et vérifie si l'utilisateur a exprimé un cas d'usage intéressant.
Si l'utilisateur n'a pas réussi, dit le.
Si l'utilisateur a réussi, résume le cas d'usage. 

Sort le résultat en texte pur, sans markdown : propose plutôt une mise en page à partir d'espaces blancs 
et de tirets.
"""

In [134]:
toutes_les_conversations  = []
for session, sub_df in df.groupby("session_id"):
    print(session)
    sub_df = sub_df.sort_values("timestamp")
    conversation = list(zip(sub_df.question, sub_df.responsemessages))
    dialogue = ""
    for user, agent in  conversation:
        dialogue += f"user    : {user}\n"
        #dialogue += f"chatbot : {agent}\n"
    
    to_print = []
    for _ in generate([prompt, dialogue]):
        to_print.append(_.text)
    print(dialogue)
    toutes_les_conversations.append(dialogue)
    print("".join(to_print))


dfMessenger-0ae41a4d-6864-49f2-bf08-99d4d803df98
user    : bONJOUR
user    : Je suis responsable du système d'imformation décisionnel du département
user    : oui. sans doute pour améliorer l'analyse de données 
user    : Je demanderai à l'IA à m'aider à trouver le reprséentations graphiques ou textuelles les plus adaptées pour présenter certains résultats
user    : Cela permettrait d'automatiser la génération de tableaux de bords
user    : Cela varie en fonction de la complexité des représentation à construire mais surtout de la difficulté à trouver les données les plus pertinentes.
user    : un temps important. Mais l'IA devra m'aider pour identifier les données incomplétes, de mauvaise qualité
user    : cela permet de connaitre les axes d'améliorations 

      - L'utilisateur a exprimé un cas d'usage intéressant pour l'IA générative.
      - L'utilisateur souhaite utiliser l'IA pour automatiser la génération de tableaux de bord.
      - L'IA aiderait à identifier les représentations

In [140]:
concatenation = ""
for i in range(len(toutes_les_conversations)):
    print(i)
    conv = toutes_les_conversations[i]
    concatenation += f"<conversation_{i}>{conv}</conversation_{i}>"
concatenation

0
1
2
3
4
5


"<conversation_0>user    : bONJOUR\nuser    : Je suis responsable du système d'imformation décisionnel du département\nuser    : oui. sans doute pour améliorer l'analyse de données \nuser    : Je demanderai à l'IA à m'aider à trouver le reprséentations graphiques ou textuelles les plus adaptées pour présenter certains résultats\nuser    : Cela permettrait d'automatiser la génération de tableaux de bords\nuser    : Cela varie en fonction de la complexité des représentation à construire mais surtout de la difficulté à trouver les données les plus pertinentes.\nuser    : un temps important. Mais l'IA devra m'aider pour identifier les données incomplétes, de mauvaise qualité\nuser    : cela permet de connaitre les axes d'améliorations \n</conversation_0><conversation_1>user    : Bonjour\nuser    : directeur adjoint\nuser    : piloter, manager, décider, conseiller les directions sur les usages numériques\nuser    : oui\nuser    : aider au choix d'un logiciel métier\nuser    : il faudrait qu

In [153]:
print(dialogue)

user    : bonjour



In [155]:
prompt = "voici des conversations, sors moi les typologies de cas d'usage de l'IA qui émergent"

In [156]:
for _ in generate([prompt, concatenation]):
    print(_.text)

##
 Typologies de cas d'usage de l'IA dans les conversations :


D'après les conversations fournies, on peut identifier plusieurs typologies de cas d
'usage de l'IA :

**1. Analyse et visualisation de données :**

* **Conversation 0 :** L'utilisateur souhaite utiliser l
'IA pour améliorer l'analyse de données et la génération de tableaux de bord. Il s'agit d'automatiser la recherche des données pertinentes et la
 création de représentations graphiques ou textuelles adaptées.
* **Avantages :** Gain de temps, amélioration de la prise de décision grâce à une meilleure compréhension des données, identification des axes d'amélioration.

**2. Analyse de
 documents et synthèse d'informations :**

* **Conversation 1 :** L'utilisateur recherche une aide pour analyser des documentations de logiciels et pour synthétiser des informations contenues dans sa boîte mail.
* **Avantages
 :** Gain de temps, efficacité accrue, meilleure prise de décision grâce à une information synthétique et pertinente.


##
 Cas d'usage de l'IA identifiés dans les conversations :

**Analyse
 de données et reporting :**

* **Identifier les représentations graphiques et text
uelles optimales pour présenter des résultats** (Conversation 0) : L'IA peut analyser les données et suggérer les visualisations les plus pertinentes pour faciliter
 la compréhension et la prise de décision.
* **Automatiser la génération de tableaux de bord** (Conversation 0) : L'IA peut créer des
 tableaux de bord dynamiques et personnalisés, actualisés en temps réel, pour un suivi efficace des indicateurs clés.
* **Identifier les données incomplètes ou de mauvaise qualité** (Conversation 0) : L'IA peut analyser
 la qualité des données et détecter les anomalies ou les erreurs, améliorant ainsi la fiabilité des analyses et des rapports.

**Aide à la décision et gain de temps :**

* **Analyser des documentations pour comparer les fonctionnalités
 des logiciels** (Conversation 1) : L'IA peut extraire et synthétiser les informations clés de documents complexes, facilitant le choix d'un logiciel adapté aux besoins de l'entreprise.
* **Synthétiser des informations provenant de sources multiples, comme les emails** (Conversation 
1) : L'IA peut analyser et résumer un grand volume d'informations, permettant aux utilisateurs de se concentrer sur les points essentiels.

**Gestion des stocks et des normes :**

* **Gérer les stocks de denrées alimentaires** (Conversation 2) : L'IA peut optimiser
 les niveaux de stock, prévoir les besoins en fonction de la demande et minimiser le gaspillage.
* **Suivre et améliorer les protocoles HACCP** (Conversation 2) : L'IA peut automatiser le suivi des normes d'hygiène et de sécurité alimentaire, réduisant les risques et améliorant
 la conformité.

**Autres cas d'usage potentiels :**

* **Améliorer les modes opératoires des métiers** (Conversation 3) : L'IA peut analyser les processus existants et identifier des opportunités d'optimisation et d'automatisation.
* **Prise de rendez
-vous** (Conversation 3) : L'IA peut automatiser la planification de rendez-vous en fonction des disponibilités et des préférences des utilisateurs. 


**Remarques :**

* Les conversations ne fournissent pas suffisamment d'informations pour évaluer la faisabilité ou la complexité de chaque cas d
'usage. 
* Il est important de considérer les aspects éthiques et les biais potentiels de l'IA lors de la mise en œuvre de ces solutions. 







reponses = generate([prompt, concatenation])
​
for reponse in reponses:
    print.append(reponse.text)

In [149]:
reponses = generate([prompt, concatenation])

for reponse in reponses:
    print.append(reponse.text)

AttributeError: 'builtin_function_or_method' object has no attribute 'append'

In [165]:
!pwd

/home/jupyter/vertex-ai-demo
